In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [5]:
#@title Helper functions
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

## Dataset

In [6]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


## Normalize the data

In [9]:
norm_l = tf.keras.layers.Normalization(axis = -1)
norm_l.adapt(X)
Xn = norm_l(X) # Normalized X

In [11]:
# Tile data to increase training set size and decrease the number of training epochs
X_train = np.tile(Xn, (1000, 1))
Y_train = np.tile(Y, (1000, 1))
print(X_train.shape, Y_train.shape)

(200000, 2) (200000, 1)


## Tensorflow Model

In [13]:
tf.random.set_seed(1234)
model = tf.keras.models.Sequential ([
    tf.keras.Input(shape = (2, )),
    tf.keras.layers.Dense(3, activation = 'sigmoid', name = 'layer1'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'layer2')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
W1, b1 = model.get_layer('layer1').get_weights()
W2, b2 = model.get_layer('layer2').get_weights()
print(f"W1 {W1.shape}:", W1, f"\nb1 {b1.shape}:", b1)
print(f"W2 {W2.shape}:", W2, f"\nb2 {b2.shape}:", b2)

W1 (2, 3): [[ 1.0463135  -0.40092605  0.42326307]
 [-0.15899467  0.25358236 -0.82428575]] 
b1 (3,): [0. 0. 0.]
W2 (3, 1): [[-0.35933965]
 [-0.9581658 ]
 [-0.44844812]] 
b2 (1,): [0.]


### Model compile and Model fit